# Econ 122 HW 1
### Oscar Scholin

In [1]:
# imports
import pyreadstat as prs
from helper import *

# read in list of files
# files = [f'data/morg79.dta'] + [f'data/morg{j}{l}.dta' for j in [8, 9, 0, 1] for l in range(0, 10)] + [f'data/morg20.dta']
files = ['data/morg79.dta','data/morg19.dta' ]

## Problem 1

In [2]:
# for each year, log hourly wage, % < high school, % with college, years edu, potential exp, potential exp squared, sample size
# do this for 3 criteria: men and women age 24-65, men age 24-65, women age 24-65
# weight based on hrearnwt = dataframe['earnwt'] * dataframe['uhours']

# define dictionary of dataframes
dfs = {}

for file in files:
    print(file)
    # create dataframe
    df, meta = prs.read_dta(file)
    # create year variable
    # split .dta, take last two characters, convert to int
    year = int(file.split('.')[0][-2:])
    dfs[year] = df

print(dfs.keys())

data/morg79.dta
data/morg19.dta
dict_keys([79, 19])


In [3]:
dfs[19].columns

Index(['hhid', 'intmonth', 'hurespli', 'hrhtype', 'minsamp', 'hrlonglk',
       'hrsample', 'hrhhid2', 'serial', 'hhnum', 'stfips', 'cbsafips',
       'county', 'centcity', 'smsastat', 'icntcity', 'smsa04', 'relref95',
       'age', 'spouse', 'sex', 'grade92', 'race', 'ethnic', 'lineno', 'famnum',
       'pfamrel', 'marital', 'prpertyp', 'penatvty', 'pemntvty', 'pefntvty',
       'prcitshp', 'prcitflg', 'peinusyr', 'selfproxy', 'lfsr94', 'absent94',
       'uhourse', 'reason94', 'hourslw', 'laydur', 'dwrsn', 'why3594',
       'prunedur', 'untype', 'ftpt94', 'class94', 'agri', 'eligible', 'otc',
       'ernpdh2', 'paidhre', 'earnhre', 'earnwke', 'unionmme', 'unioncov',
       'schenr', 'studftpt', 'schlvl', 'earnwt', 'weight', 'chldpres',
       'ownchild', 'I25d', 'I25c', 'I25a', 'I25b', 'qstnum', 'occurnum', 'ged',
       'gedhigr', 'yrcoll', 'grprof', 'gr6cor', 'ms123', 'cmpwgt', 'ind02',
       'occ2012', 'occ002', 'vet1', 'vet2', 'vet3', 'vet4', 'linedad',
       'linemom', 'recnum

In [4]:
# now we need to read in the inflation data and create df
# inflation = [33.804,34.728,35.590,36.451,37.545,38.464,39.364,40.338,41.384,42.078,42.772,43.427,43.979,44.402,45.103,45.601,45.978,46.398,47.008,47.318,47.830,48.293,48.666,48.966,49.544,49.945,50.337,50.688,51.047,50.993,51.263,51.573,52.061,52.564,53.062,53.520,53.943,54.539,55.211,55.769,56.410,57.170,57.507,57.959,58.799,59.333,60.087,60.882,61.202,61.536,61.954,62.404,62.795,63.212,63.615,64.059,64.441,64.874,65.155,65.531,65.766,66.133,66.607,66.920,67.247,67.639,67.915,68.214,68.593,69.052,69.346,69.819,70.127,70.303,70.488,70.710,70.715,70.843,71.062,71.249,71.390,71.796,72.191,72.629,73.219,73.568,74.042,74.460,75.012,75.363,75.401,75.432,75.584,76.145,76.539,76.896,77.484,77.562,78.073,78.457,79.062,79.594,79.985,80.669,81.138,81.650,82.532,83.189,83.621,84.355,84.962,84.822,85.597,86.324,86.812,87.694,88.408,89.269,90.221,88.781,88.181,88.532,89.142,89.830,90.177,90.317,90.490,91.070,91.835,92.738,93.167,93.475,94.094,94.321,94.595,95.126,95.459,95.508,95.901,96.254,96.696,97.129,97.394,97.265,96.829,97.312,97.565,97.490,97.538,98.157,98.496,98.947,99.524,99.724,100.076,100.676,101.380,101.912,102.254,102.642,102.877,103.422,103.674,104.080]
# # divide into groups of 4 for each quarter within a year
# inflation = [inflation[i:i+4] for i in range(0, len(inflation), 4)]
# inflation
inflation = [[33.804,34.728,35.590,36.451], [102.877, 103.422, 103.674, 104.080]]
inflation_2019 = [102.877, 103.422, 103.674, 104.080]

In [5]:
# now that data is loaded in dictionary, can create tables
# add to each df the new variables
for j, year in enumerate(dfs.keys()):
    df = dfs[year]
    # create new variables

    # make sure to compare the right row entries so the division makes sense
    if 'uhours' in df.columns:
        # drop all values =< 0
        df.loc[df['uhours'] <= 0, 'uhours'] = np.nan
        df['hrwage'] = df['earnwke'] / df['uhours']
        dfs[year]['hrearnwt'] = dfs[year]['earnwt'] * dfs[year]['uhours']
    elif 'uhourse' in df.columns:
        # drop all values =< 0
        df.loc[df['uhourse'] <= 0, 'uhourse'] = np.nan
        df['hrwage'] = df['earnwke'] / df['uhourse']
        dfs[year]['hrearnwt'] = dfs[year]['earnwt'] * dfs[year]['uhourse']
    
    ## first adjust for inflation for each quarter ##
    # get the row from the inflation list
    inflation_row = inflation[j]
    realhrwage = df['hrwage'] 
    # if month between 1 and 3, then multiply by 100/33.804
    realhrwage = np.where(df['intmonth'] <= 3, realhrwage * 100/inflation_row[0], realhrwage)
    # if month between 4 and 6, then multiply by 100/34.728
    realhrwage = np.where((df['intmonth'] > 3) & (df['intmonth'] <= 6), realhrwage * inflation_2019[0]/inflation_row[1], realhrwage)
    # if month between 7 and 9, then multiply by 100/35.590
    realhrwage = np.where((df['intmonth'] > 6) & (df['intmonth'] <= 9), realhrwage * inflation_2019[1]/inflation_row[2], realhrwage)
    # if month between 10 and 12, then multiply by 100/36.451
    realhrwage = np.where((df['intmonth'] > 9) & (df['intmonth'] <= 12), realhrwage * inflation_2019[2]/inflation_row[3], realhrwage)
    
    df['realhrwage'] = realhrwage
    # remove values < 2 and > 250
    df.loc[df['realhrwage'] < 2, 'realhrwage'] = np.nan
    df.loc[df['realhrwage'] > 250, 'realhrwage'] = np.nan


    ## get educ; treat gradeat vs grade92 separately ##
    # if grade32 exists, then need to convert that; otheerwise have to deal with gradeat and gradecp
    # create educ variable
    if 'grade92' in df.columns:
        df['educ'] = df['grade92']
        
        df.loc[df['grade92'] == 32, 'educ'] = 2.5
        df.loc[df['grade92'] == 33, 'educ'] = 5.5
        df.loc[df['grade92'] == 34, 'educ'] = 7.5
        df.loc[df['grade92'] == 35, 'educ'] = 9
        df.loc[df['grade92'] == 36, 'educ'] = 10
        df.loc[df['grade92'] == 37, 'educ'] = 11
        df.loc[df['grade92'] == 38, 'educ'] = 12
        df.loc[df['grade92'] == 39, 'educ'] = 12
        df.loc[df['grade92'] == 40, 'educ'] = 13
        df.loc[df['grade92'] == 41, 'educ'] = 14
        df.loc[df['grade92'] == 42, 'educ'] = 14
        df.loc[df['grade92'] == 43, 'educ'] = 16
        df.loc[df['grade92'] == 44, 'educ'] = 18
        df.loc[df['grade92'] == 45, 'educ'] = 18
        df.loc[df['grade92'] == 46, 'educ'] = 18

    else:
        df['educ'] = df['gradeat']
        # if gradecp is 0, then subtract 1
        df.loc[df['gradecp'] == 0, 'educ'] = df['educ'] - 1
    

    logwage = dfs[year]['realhrwage'].to_numpy()
    # conver to float
    logwage = logwage.astype(float)
    df['logwage'] = np.log(logwage)
    df['lths'] = dfs[year]['educ'] < 12
    df['college'] = dfs[year]['educ'] >= 16
    df['exp'] = dfs[year]['age'] - dfs[year]['educ'] - 6
    df['exp2'] = dfs[year]['exp']**2

    # reset key value
    dfs[year] = df

/var/folders/zx/ck7rkn5j0_zfqqnb92k3668r0000gn/T/ipykernel_57871/1499423517.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['grade92'] == 32, 'educ'] = 2.5


In [6]:
summarize(dfs[19]['logwage'])

Num observations:  145671
Mean:  3.056409905460303
Median:  3.0004645871537545
Standard Deviation:  0.5885429473163541
Variance:  0.3463828008358208
1st percentile:  1.787850995582845
5th percentile:  2.1919409718134273
10th percentile:  2.3150431736248747
25th percentile:  2.6351488559700487
50th percentile:  3.0004645871537545
75th percentile:  3.468143647577054
90th percentile:  3.870328636316113
95th percentile:  4.058009355387689
99th percentile:  4.27435706006941
Skewness:  0.11876136435305619
Kurtosis:  -0.20088111610288362
Min:  0.6998961554714015
Max:  5.519027262703516


### Answer to Question 1

In [7]:
# compute weighted averages for each logwage, % less than high school, % with college, years of education, potential experience, potential experience squared, sample size
# do this for each year
for year in dfs.keys():
    df = dfs[year]
    # list of separate conditions to run
    conditions = [(df['age'] >= 24) & (df['age'] <= 65), (df['sex'] ==1) & (df['age'] >= 24) & (df['age'] <= 65), (df['sex'] ==2) & (df['age'] >= 24) & (df['age'] <= 65)]
    conditions_names = ['24-65', 'M 24-65', 'W 24-65']
    # build dataframe where these conditions form the rows, the cols are the avgs
    # create new dataframe
    df2 = pd.DataFrame()
    for i, condition in enumerate(conditions):
        df_condition = df[condition]
        # df_condition = df_condition.dropna()
        weight = df_condition['hrearnwt']
    
        # drop all weight entries with nan
        weight = weight.dropna()

        # drop all rows where weight is nan
        df_condition = df_condition.dropna(subset=['hrearnwt'])

        # drop all rows where logwage is nan
        df_condition = df_condition.dropna(subset=['logwage'])

        # drop weights where logwage is nan
        weight = weight[df_condition.index]

        # get weighted avg of each
        logwage_wawg = np.average(df_condition['logwage'], weights=weight)
        lths_wawg = np.average(df_condition['lths'], weights=weight)
        college_wawg = np.average(df_condition['college'], weights=weight)
        educ_wawg = np.average(df_condition['educ'], weights=weight)
        exp_wawg = np.average(df_condition['exp'], weights=weight)
        exp2_wawg = np.average(df_condition['exp2'], weights=weight)
        sample_size = df_condition.shape[0]

        # create new dataframe
        df2[conditions_names[i]] = [logwage_wawg, lths_wawg, college_wawg, educ_wawg, exp_wawg, exp2_wawg, sample_size]

    # add labels to the dataframe
    df2.index = ['logwage', 'lths', 'college', 'educ', 'exp', 'exp2', 'sample_size']
    # save as csv
    df2.to_csv(f'data/df2_{year}.csv')
    

## Question 3

In [8]:
'lfsr94' in dfs[19].columns

True

In [9]:
# redo question 1 for additional constraint that esr == 1
# compute weighted averages for each logwage, % less than high school, % with college, years of education, potential experience, potential experience squared, sample size
# do this for each year
for year in dfs.keys():
    df = dfs[year]
    # list of separate conditions to run
    conditions = [(df['age'] >= 24) & (df['age'] <= 65), (df['sex'] ==1) & (df['age'] >= 24) & (df['age'] <= 65), (df['sex'] ==2) & (df['age'] >= 24) & (df['age'] <= 65)]
    conditions_names = ['24-65', 'M 24-65', 'W 24-65']
    # build dataframe where these conditions form the rows, the cols are the avgs
    # create new dataframe
    df2 = pd.DataFrame()
    for i, condition in enumerate(conditions):
        if year == 19:
            condition = condition & (df['lfsr94'] == 1)
        else:
            condition = condition & (df['esr'] == 1)
        df_condition = df[condition]
        # df_condition = df_condition.dropna()
        weight = df_condition['hrearnwt']
    
        # drop all weight entries with nan
        weight = weight.dropna()

        # drop all rows where weight is nan
        df_condition = df_condition.dropna(subset=['hrearnwt'])

        # drop all rows where logwage is nan
        df_condition = df_condition.dropna(subset=['logwage'])

        # drop weights where logwage is nan
        weight = weight[df_condition.index]

        # get weighted avg of each
        logwage_wawg = np.average(df_condition['logwage'], weights=weight)
        lths_wawg = np.average(df_condition['lths'], weights=weight)
        college_wawg = np.average(df_condition['college'], weights=weight)
        educ_wawg = np.average(df_condition['educ'], weights=weight)
        exp_wawg = np.average(df_condition['exp'], weights=weight)
        exp2_wawg = np.average(df_condition['exp2'], weights=weight)
        sample_size = df_condition.shape[0]

        # create new dataframe
        df2[conditions_names[i]] = [logwage_wawg, lths_wawg, college_wawg, educ_wawg, exp_wawg, exp2_wawg, sample_size]

    # add labels to the dataframe
    df2.index = ['logwage', 'lths', 'college', 'educ', 'exp', 'exp2', 'sample_size']
    # save as csv
    df2.to_csv(f'data/df2_{year}_working.csv')
    

## Question 2

In [10]:
# run WLS for each year with each condition
for year in dfs.keys():
    df = dfs[year]
    # list of separate conditions to run
    if 'esr' in df.columns:
        conditions = [(df['age'] >= 24) & (df['age'] <= 65) & (df['esr']==1), (df['sex'] ==1) & (df['age'] >= 24) & (df['age'] <= 65)& (df['esr']==1), (df['sex'] ==2) & (df['age'] >= 24) & (df['age'] <= 65)& (df['esr']==1)]
    elif 'pemlr' in df.columns:
        conditions = [(df['age'] >= 24) & (df['age'] <= 65) & (df['pemlr']==1), (df['sex'] ==1) & (df['age'] >= 24) & (df['age'] <= 65)& (df['pemlr']==1), (df['sex'] ==2) & (df['age'] >= 24) & (df['age'] <= 65)& (df['pemlr']==1)]

    # get dataset for regression
    X_cols = ['educ', 'exp', 'exp2']
    y_col = 'logwage'
    weight = 'hrearnwt'

    for j, condition in enumerate(conditions):
        X, y, w = prepare_data(df, condition, X_cols, y_col, weight)

        # run WLS
        print('Condition ', j, 'Year ', year)
        run_WLS(X, y, w, print_summary=True)
        print('\n \n \n')

Condition  0 Year  79
                            WLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.133
Model:                            WLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     4337.
Date:                Sun, 11 Feb 2024   Prob (F-statistic):               0.00
Time:                        23:21:49   Log-Likelihood:                -87067.
No. Observations:              120587   AIC:                         1.741e+05
Df Residuals:                  120583   BIC:                         1.742e+05
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7793      0.0

/Users/oscarscholin/Desktop/Pomona/Senior_Year/Spring2024/Econ122/Econ122/helper.py:75: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = dataframe[X_cols][condition]
/Users/oscarscholin/Desktop/Pomona/Senior_Year/Spring2024/Econ122/Econ122/helper.py:75: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = dataframe[X_cols][condition]


                            WLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.253
Model:                            WLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     3755.
Date:                Sun, 11 Feb 2024   Prob (F-statistic):               0.00
Time:                        23:21:49   Log-Likelihood:                -53964.
No. Observations:               62784   AIC:                         1.079e+05
Df Residuals:                   62780   BIC:                         1.080e+05
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4026      0.018     80.100      0.0

/Users/oscarscholin/Desktop/Pomona/Senior_Year/Spring2024/Econ122/Econ122/helper.py:75: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = dataframe[X_cols][condition]


## Question 5

In [11]:
from importlib import reload
import helper
reload(helper)

<module 'helper' from '/Users/oscarscholin/Desktop/Pomona/Senior_Year/Spring2024/Econ122/Econ122/helper.py'>

In [12]:
# oaxaca blinder decomposition on men vs female
for year in dfs.keys():
    df = dfs[year]

    X_cols = ['educ', 'exp', 'exp2']
    y_col = 'logwage'
    weight = 'hrearnwt'
    condition1 = df['sex'] == 1
    condition2 = df['sex'] == 2

    X1, y1, w1 = prepare_data(df, condition1, X_cols, y_col, weight)
    X2, y2, w2 = prepare_data(df, condition2, X_cols, y_col, weight)

    print('Year ', year)
    print('Men vs Women')
    oaxaca_blinder(X1, X2, y1, y2, w1, w2)
    print('\n')
    print('Women vs Men')
    oaxaca_blinder(X2, X1, y2, y1, w2, w1)
    print('\n')
    print('Overall vs Men')
    X_o, y_o, w_o = prepare_data(df, condition=(df['sex']==1) | (df['sex']==2), X_cols=X_cols, Y_col=y_col, W_col=weight)
    oaxaca_blinder(X_o, X1, y_o, y1, w_o, w1)
    print('\n')
    print('Overall vs Women')
    oaxaca_blinder(X_o, X2, y_o, y2, w_o, w2)
    print('\n \n \n')

Year  79
Men vs Women
Mean diff:  const     0.000000
educ      0.000707
exp       0.733723
exp2     24.913193
dtype: float64
Coeff1:  [ 1.72714581e+00  6.23480967e-02  4.27825967e-02 -7.06319169e-04]
Coeff2:  [ 1.48247525e+00  6.89251253e-02  2.21795085e-02 -3.86516723e-04]
Explained component: 0.013837995167081129
Unexplained component: 0.3547928065842964
Total gap: 0.3686308017513775


Women vs Men
Mean diff:  const     0.000000
educ     -0.000707
exp      -0.733723
exp2    -24.913193
dtype: float64
Coeff1:  [ 1.48247525e+00  6.89251253e-02  2.21795085e-02 -3.86516723e-04]
Coeff2:  [ 1.72714581e+00  6.23480967e-02  4.27825967e-02 -7.06319169e-04]
Explained component: -0.006692990494700658
Unexplained component: -0.361937811256677
Total gap: -0.3686308017513777


Overall vs Men
Mean diff:  const     0.000000
educ     -0.000312
exp      -0.323789
exp2    -10.994098
dtype: float64
Coeff1:  [ 1.62678101e+00  6.49308931e-02  3.52059345e-02 -5.88046660e-04]
Coeff2:  [ 1.72714581e+00  6.234

## Question 10

In [13]:
'vet4' in dfs[19].columns

True

In [14]:
summarize(dfs[79]['veteran'])

Num observations:  154541
Mean:  4.744734407050556
Median:  6.0
Standard Deviation:  1.829572136401636
Variance:  3.347334202297246
1st percentile:  1.0
5th percentile:  1.0
10th percentile:  1.0
25th percentile:  3.0
50th percentile:  6.0
75th percentile:  6.0
90th percentile:  6.0
95th percentile:  6.0
99th percentile:  6.0
Skewness:  -1.0186873753333952
Kurtosis:  -0.6018548411151272
Min:  1.0
Max:  6.0


In [15]:
dfs[79].loc[dfs[79]['veteran'] != 6]

,minsamp,intmonth,hhid,state,smsarank,hhnum,activlwr,hourslw,reasonlw,absentlw,...,dind,hrwage,hrearnwt,realhrwage,educ,logwage,lths,college,exp,exp2
0,4,1,003001221503,93,0,1,1,40,NaN,NaN,...,19,6.0,282229.609375,17.749379,10,2.876351,True,False,24,576
1,4,1,003001221503,93,0,1,1,40,NaN,NaN,...,41,7.0,268433.203125,20.707609,12,3.030501,False,False,18,324
4,4,1,003001222503,93,0,1,1,40,NaN,NaN,...,19,5.175,295139.199219,15.308839,12,2.728430,False,False,22,484
7,4,1,003001223503,93,0,1,1,40,NaN,NaN,...,1,5.175,268433.203125,15.308839,12,2.728430,False,False,21,441
11,8,1,003001231507,93,0,1,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10,NaN,True,False,60,3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328392,8,12,805398119712,94,0,2,7,NaN,NaN,NaN,...,33,NaN,NaN,NaN,11,NaN,True,False,2,4
328394,8,12,903002711002,94,0,1,7,NaN,NaN,NaN,...,46,NaN,NaN,NaN,0,NaN,True,False,68,4624
328398,8,12,903002713002,94,0,1,4,NaN,NaN,NaN,...,46,NaN,NaN,NaN,0,NaN,True,False,75,5625
328400,8,12,903002713002,94,0,1,4,NaN,NaN,NaN,...,3,NaN,NaN,NaN,12,NaN,False,False,3,9


In [16]:
dfs[19].loc[(dfs[19]['vet1'] != 1) & (dfs[19]['vet1'] != 2) & (dfs[19]['vet1'] != 3) & (dfs[19]['vet1'] != 4) & (dfs[19]['vet1'] != 5) & (dfs[19]['vet1'] != 6) & (dfs[19]['vet1'] != 7) & (dfs[19]['vet1'] != 8) & (dfs[19]['vet1'] != 9)]

,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,dind02,hrwage,hrearnwt,realhrwage,educ,logwage,lths,college,exp,exp2
0,000110327856469,1,1,7,4,2,0911,09111,1,1,...,22,14.0,272808.0624,13.608484,13.0,2.610693,False,False,3.0,9.0
1,000110339935453,1,1,4,8,2,0701,07011,1,1,...,41,20.91325,261633.664,20.328402,13.0,3.012019,False,False,35.0,1225.0
2,000110339935453,1,1,4,8,2,0701,07011,1,1,...,46,7.65,126540.7,7.436064,10.0,2.006342,True,False,1.0,1.0
3,000110339935453,1,1,4,8,2,0701,07011,1,1,...,46,7.65,126540.7,7.436064,10.0,2.006342,True,False,1.0,1.0
4,000110339935453,1,1,4,8,2,0701,07011,1,1,...,22,10.0,168922.5765,9.720346,13.0,2.274221,False,False,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291384,710106595598086,12,1,4,8,2,0901,09011,1,1,...,32,14.5,52832.727,14.443438,12.0,2.670240,False,False,15.0,225.0
291385,802505349610555,12,1,1,4,2,1001,10011,1,1,...,47,NaN,47584.7808,NaN,12.0,NaN,False,False,37.0,1369.0
291387,876944601471509,12,1,6,4,2,1011,10111,1,1,...,23,NaN,NaN,NaN,13.0,NaN,False,False,41.0,1681.0
291388,905527030610215,12,2,6,4,2,1001,10011,1,1,...,4,17.0,61429.452,16.933686,13.0,2.829305,False,False,3.0,9.0


In [17]:
dfs[19]

,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,dind02,hrwage,hrearnwt,realhrwage,educ,logwage,lths,college,exp,exp2
0,000110327856469,1,1,7,4,2,0911,09111,1,1,...,22,14.0,272808.0624,13.608484,13.0,2.610693,False,False,3.0,9.0
1,000110339935453,1,1,4,8,2,0701,07011,1,1,...,41,20.91325,261633.664,20.328402,13.0,3.012019,False,False,35.0,1225.0
2,000110339935453,1,1,4,8,2,0701,07011,1,1,...,46,7.65,126540.7,7.436064,10.0,2.006342,True,False,1.0,1.0
3,000110339935453,1,1,4,8,2,0701,07011,1,1,...,46,7.65,126540.7,7.436064,10.0,2.006342,True,False,1.0,1.0
4,000110339935453,1,1,4,8,2,0701,07011,1,1,...,22,10.0,168922.5765,9.720346,13.0,2.274221,False,False,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291385,802505349610555,12,1,1,4,2,1001,10011,1,1,...,47,NaN,47584.7808,NaN,12.0,NaN,False,False,37.0,1369.0
291386,802505349610555,12,1,1,4,2,1001,10011,1,1,...,21,35.0,58052.112,34.86347,16.0,3.551440,False,True,31.0,961.0
291387,876944601471509,12,1,6,4,2,1011,10111,1,1,...,23,NaN,NaN,NaN,13.0,NaN,False,False,41.0,1681.0
291388,905527030610215,12,2,6,4,2,1001,10011,1,1,...,4,17.0,61429.452,16.933686,13.0,2.829305,False,False,3.0,9.0


In [18]:
# rerun question 7 but including own children
for year in dfs.keys():
    df = dfs[year]

    X_cols = ['educ', 'exp', 'exp2']
    y_col = 'logwage'
    weight = 'hrearnwt'
    if 'veteran' in df.columns:
        condition1 = df['sex'] == 1 & (df['veteran'] != 6) # male veteran
        condition2 = df['sex'] == 1 & (df['veteran'] == 6) # male non-veteran
    else:
        non_veteran = df['sex'] == 1 & ((df['vet1'] != 1) & (df['vet1'] != 2) & (df['vet1'] != 3) & (df['vet1'] != 4) & (df['vet1'] != 5) & (df['vet1'] != 6) & (df['vet1'] != 7) & (df['vet1'] != 8) & (df['vet1'] != 9))
        
    X1, y1, w1 = prepare_data(df, condition1, X_cols, y_col, weight)
    X2, y2, w2 = prepare_data(df, condition2, X_cols, y_col, weight)

    print('Year ', year)
    print('Male Veteran vs Non-Veteran')
    oaxaca_blinder(X1, X2, y1, y2, w1, w2)
    print('\n \n \n')


Year  79
Male Veteran vs Non-Veteran
Mean diff:  const      0.000000
educ       0.332356
exp       11.874205
exp2     414.758154
dtype: float64
Coeff1:  [ 1.89126694e+00  5.63851074e-02  3.66231141e-02 -6.01945322e-04]
Coeff2:  [ 1.69241477e+00  6.43346870e-02  4.31766408e-02 -7.19790447e-04]
Explained component: 0.203948572316797
Unexplained component: 0.0556808375985334
Total gap: 0.2596294099153304

 
 

Year  19
Male Veteran vs Non-Veteran
Mean diff:  const    0.000000
educ    -0.006657
exp     -0.100664
exp2    -4.499779
dtype: float64
Coeff1:  [ 1.40168928e+00  9.58388711e-02  2.91808721e-02 -4.46321876e-04]
Coeff2:  [ 1.38250270e+00  9.60462425e-02  3.10872547e-02 -4.85024289e-04]
Explained component: -0.0015670833084194665
Unexplained component: 0.0010630856738693684
Total gap: -0.0005039976345500982

 
 



/Users/oscarscholin/Desktop/Pomona/Senior_Year/Spring2024/Econ122/Econ122/helper.py:75: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = dataframe[X_cols][condition]
/Users/oscarscholin/Desktop/Pomona/Senior_Year/Spring2024/Econ122/Econ122/helper.py:75: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = dataframe[X_cols][condition]
